In [3]:
import pathlib
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [4]:
DATA_PATH = pathlib.Path("UCI_Credit_Card.csv")
if not DATA_PATH.exists():
    raise FileNotFoundError(
        "Arquivo 'UCI_Credit_Card.csv' não encontrado. Coloque-o na mesma pasta "
        "ou ajuste DATA_PATH.")

print("📊  Carregando dados…")
df = pd.read_csv(DATA_PATH)
print(f"✔️  Linhas: {len(df):,}; Colunas: {len(df.columns)}")

TARGET_COL = "default.payment.next.month"  # 1: sim (default), 0: não
ID_COL = "ID"   

📊  Carregando dados…
✔️  Linhas: 30,000; Colunas: 25


In [5]:
X = df.drop(columns=[ID_COL, TARGET_COL])
y = df[TARGET_COL]
FEATURE_NAMES = X.columns.tolist()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [10]:
xgb_params = dict(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
)

In [11]:
print("🚀  Treinando XGBoost…")
model = xgb.XGBClassifier(**xgb_params)
model.fit(X_train, y_train)

🚀  Treinando XGBoost…


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [12]:
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f"🎯  AUC no teste: {auc:.4f}")

🎯  AUC no teste: 0.7748


In [9]:
print("💾  Salvando booster em JSON…")
booster = model.get_booster()
booster.save_model("credit_default_xgb.json")

print("💾  Salvando Pipeline completa (opcional)…")
try:
    import joblib

    joblib.dump(model, "credit_default_xgb.pkl")
except ImportError:
    print("⚠️  joblib não instalado; pulei .pkl. Use 'pip install joblib' se quiser.")

print("✅  Concluído!")

💾  Salvando booster em JSON…
💾  Salvando Pipeline completa (opcional)…
✅  Concluído!
